# Supervised Learning

In [3]:
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import glob

### Q2.1 Classic Machine Learning Methods (5 Pts)
#### Q2.1 1.

In [4]:
# Retrieving the X matrix
df = pd.read_parquet('final-data/final-set-a.parquet')
df = df.fillna(0)
X = df.groupby("RecordID").last(numeric_only=True).reset_index()
X = X.drop(columns=["RecordID"])
X = X[sorted(X.columns)]

In [5]:
# Retrieving the label vector
y_df = pd.read_parquet('processed-data/processed-outcomes-a.parquet')
y = y_df["In-hospital_death"].to_numpy().flatten()
print(y.sum())
print(len(y))

554
4000


In [6]:
# Logistic Regression
model1 = LogisticRegression(max_iter=500, class_weight='balanced')
model1.fit(X,y)

# Random Forest
model2 = RandomForestClassifier(
    n_estimators=600,
    min_samples_split=5,    
    min_samples_leaf=2,         
    class_weight='balanced',
    random_state=42,        
)
model2.fit(X,y)

# KNN
model3 = KNeighborsClassifier(
    n_neighbors=150,
    weights='distance',
    metric='manhattan',
)
model3.fit(X,y)

KNeighborsClassifier(metric='manhattan', n_neighbors=150, weights='distance')

In [7]:
# Test set C performance

# Loading test set C
df = pd.read_parquet('final-data/final-set-c.parquet')
df = df.fillna(0)
df = df.drop(columns=["ICUType"])
X_test = df.groupby("RecordID").last(numeric_only=True).reset_index()
X_test = X_test.drop(columns=["RecordID"])
X_test= X_test[sorted(X_test.columns)]

y_df = pd.read_parquet('processed-data/processed-outcomes-c.parquet')
y_test = y_df["In-hospital_death"].to_numpy().flatten()

y_pred1 = model1.predict_proba(X_test)[:,1]
y_pred2 = model2.predict_proba(X_test)[:,1]
y_pred3 = model3.predict_proba(X_test)[:,1]

# Calculation of AuROC and AuPRC for Logistic Regression
print("Logistic Regression results")
auroc = roc_auc_score(y_test, y_pred1)
print(f"AUROC: {auroc}")
auprc = average_precision_score(y_test, y_pred1)
print(f"AUPRC: {auprc}", end="\n\n")

# Calculation of AuROC and AuPRC for Random Forests
print("Random Forests results")
auroc = roc_auc_score(y_test, y_pred2)
print(f"AUROC: {auroc}")
auprc = average_precision_score(y_test, y_pred2)
print(f"AUPRC: {auprc}", end="\n\n")

# Calculation of AuROC and AuPRC for KNN
print("KNN results")
auroc = roc_auc_score(y_test, y_pred3)
print(f"AUROC: {auroc}")
auprc = average_precision_score(y_test, y_pred3)
print(f"AUPRC: {auprc}", end="\n\n")

Logistic Regression results
AUROC: 0.8446701955925967
AUPRC: 0.4985580595691314

Random Forests results
AUROC: 0.8554166510242646
AUPRC: 0.5120231760871147

KNN results
AUROC: 0.8367941835291762
AUPRC: 0.4887275415574222



#### Q2.1 2.

In [8]:
extraction_settings = MinimalFCParameters()

# Extracting features of concatenated training and test dataset (need to do this in one go so the feature extraction is consistent)
df_train = pd.read_parquet('final-data/final-set-a.parquet')
df_test = pd.read_parquet('final-data/final-set-c.parquet').drop(columns=["ICUType"])

print(df_train.shape)
print(df_test.shape)

df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

X_train = extract_features(df_train, column_id='RecordID', column_sort='Time', default_fc_parameters=extraction_settings, impute_function=impute)
X_test = extract_features(df_test, column_id='RecordID', column_sort='Time', default_fc_parameters=extraction_settings, impute_function=impute)

X_train= X_train[sorted(X_train.columns)]
X_test= X_test[sorted(X_test.columns)]

(196000, 42)
(196000, 42)


Feature Extraction: 100%|██████████| 20/20 [00:19<00:00,  1.05it/s]


In [9]:
y_train = pd.read_parquet('processed-data/processed-outcomes-a.parquet')["In-hospital_death"].to_numpy().flatten()
y_test = pd.read_parquet('processed-data/processed-outcomes-c.parquet')["In-hospital_death"].to_numpy().flatten()

In [10]:
# Models
# Logistic Regression
model1 = LogisticRegression(max_iter=10000, class_weight='balanced')
model1.fit(X_train,y_train)

# Random Forest
model2 = RandomForestClassifier(
    n_estimators=600,
    min_samples_split=5,    
    min_samples_leaf=2,         
    class_weight='balanced',
    random_state=42,        
)
model2.fit(X_train,y_train)

# KNN
model3 = KNeighborsClassifier(
    n_neighbors=200,
    weights='distance',
    metric='manhattan',
)
model3.fit(X_train,y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=200, weights='distance')

In [11]:
y_pred1 = model1.predict_proba(X_test)[:,1]
y_pred2 = model2.predict_proba(X_test)[:,1]
y_pred3 = model3.predict_proba(X_test)[:,1]

# Calculation of AuROC and AuPRC for Logistic Regression
print("Logistic Regression results")
auroc = roc_auc_score(y_test, y_pred1)
print(f"AUROC: {auroc}")
auprc = average_precision_score(y_test, y_pred1)
print(f"AUPRC: {auprc}", end="\n\n")

# Calculation of AuROC and AuPRC for Random Forests
print("Random Forests results")
auroc = roc_auc_score(y_test, y_pred2)
print(f"AUROC: {auroc}")
auprc = average_precision_score(y_test, y_pred2)
print(f"AUPRC: {auprc}", end="\n\n")

# Calculation of AuROC and AuPRC for KNN
print("KNN results")
auroc = roc_auc_score(y_test, y_pred3)
print(f"AUROC: {auroc}")
auprc = average_precision_score(y_test, y_pred3)
print(f"AUPRC: {auprc}", end="\n\n")

Logistic Regression results
AUROC: 0.8163296667542642
AUPRC: 0.44524358191940566

Random Forests results
AUROC: 0.8405548672898601
AUPRC: 0.46630205253654894

KNN results
AUROC: 0.817351303324949
AUPRC: 0.44402744549028644



### Q2.2 Recurrent Neural Networks (4 Pts)

LSTM approach

In [12]:
# Run in case CUDA_LAUNCH_BLOCKING error occurs
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [13]:
class PatientDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [20]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.7)
        self.bn = nn.LayerNorm(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        lstm_out, (h_n, c_n) = self.lstm(x)
        # Extracting output of last timestep
        lstm_out = lstm_out[:, -1, :]

        lstm_out = self.bn(lstm_out)

        out = self.fc(lstm_out)  
        return out
    
class BidirectionalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BidirectionalLSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.7, bidirectional=True)

        # Adjusted because of bidirectional LSTM
        self.bn = nn.LayerNorm(hidden_size * 2)  
        self.fc = nn.Linear(hidden_size * 2, output_size)
        
    def forward(self, x):
        lstm_out, (h_n, c_n) = self.lstm(x)
        # Extracting output of last timestep
        lstm_out = lstm_out[:, -1, :]

        lstm_out = self.bn(lstm_out)

        out = self.fc(lstm_out)
        return out

In [21]:
# Hyperparameters
input_size = 41
hidden_size = 256
num_layers = 8
output_size = 1
learning_rate = 0.0003
batch_size = 64
num_epochs = 10

##### NORMAL LSTM
# Test AUROC: 0.8218, Test AUPRC: 0.4636 (only set a)
""" input_size = 41
hidden_size = 256
num_layers = 8
output_size = 1
learning_rate = 0.0005
batch_size = 64
num_epochs = 10 """

# Test AUROC: 0.7828, Test AUPRC: 0.4165 (set a and set b)
""" input_size = 41
hidden_size = 256
num_layers = 8
output_size = 1
learning_rate = 0.0005
batch_size = 64
num_epochs = 9 """

##### BIDIRECTIONAL LSTM
# Test AUROC: 0.8227, Test AUPRC: 0.4557 (Only set a)
""" input_size = 41
hidden_size = 256
num_layers = 8
output_size = 1
learning_rate = 0.0003
batch_size = 64
num_epochs = 3 """

# Test AUROC: 0.7845, Test AUPRC: 0.4120 (set a and set b)
""" input_size = 41
hidden_size = 256
num_layers = 8
output_size = 1
learning_rate = 0.001
batch_size = 64
num_epochs = 4 """

' input_size = 41\nhidden_size = 256\nnum_layers = 8\noutput_size = 1\nlearning_rate = 0.001\nbatch_size = 64\nnum_epochs = 4 '

In [22]:
# NOTE: In the scaled-data set the time column is not scaled. However, due to better performance, we still scale it for this application. RecordID can be dropped, since it doesn't convey any further information and each patient is assigned one of the 4000 dimensions.
scaler = StandardScaler()

# Training Data
df = pd.read_parquet('final-data/final-set-a.parquet')
X = df.fillna(0)
X = X.groupby("RecordID").tail(49).reset_index(drop=True)
X = X.sort_values(by="RecordID", ascending=True)
X["Time"] = scaler.fit_transform(X[["Time"]])
X = X.drop(columns=["RecordID"])
X = X[sorted(X.columns)]
X = X.to_numpy()
X = X.reshape(4000,49,41)

y_df = pd.read_parquet('processed-data/processed-outcomes-a.parquet')
y = y_df["In-hospital_death"].to_numpy().flatten()

train_dataset = PatientDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


# Validation data
df = pd.read_parquet('final-data/final-set-b.parquet')
X = df.fillna(0)
X = X.groupby("RecordID").tail(49).reset_index(drop=True)
X = X.sort_values(by="RecordID", ascending=True)
X["Time"] = scaler.fit_transform(X[["Time"]])
X = X.drop(columns=["RecordID"]).drop(columns=["ICUType"])
X = X[sorted(X.columns)]
X = X.to_numpy()
X = X.reshape(4000,49,41)

y_df = pd.read_parquet('processed-data/processed-outcomes-b.parquet')
y = y_df["In-hospital_death"].to_numpy().flatten()

val_dataset = PatientDataset(X, y)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


# Test data
df = pd.read_parquet('final-data/final-set-c.parquet')
X = df.fillna(0)
X = X.groupby("RecordID").tail(49).reset_index(drop=True)
X = X.sort_values(by="RecordID", ascending=True)
X["Time"] = scaler.fit_transform(X[["Time"]])
X = X.drop(columns=["RecordID"]).drop(columns=["ICUType"])
X = X[sorted(X.columns)]
X = X.to_numpy()
X = X.reshape(4000,49,41)

y_df = pd.read_parquet('processed-data/processed-outcomes-c.parquet')
y = y_df["In-hospital_death"].to_numpy().flatten()

test_dataset = PatientDataset(X, y)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


# Full training (use after having found best model with lowest valildation error)
df = pd.read_parquet('final-data/final-set-a.parquet')
X1 = df.fillna(0)
X1 = X1.groupby("RecordID").tail(49).reset_index(drop=True)
X1 = X1.sort_values(by="RecordID", ascending=True)
X1["Time"] = scaler.fit_transform(X1[["Time"]])
X1 = X1[sorted(X1.columns)]
df = pd.read_parquet('final-data/final-set-b.parquet')
X2 = df.fillna(0)
X2 = X2.groupby("RecordID").tail(49).reset_index(drop=True)
X2 = X2.sort_values(by="RecordID", ascending=True)
X2["Time"] = scaler.fit_transform(X2[["Time"]])
X2 = X2.drop(columns=["ICUType"])
X2 = X2[sorted(X2.columns)]

X_full = pd.concat([X1, X2], axis=0).drop(columns="RecordID")

X_full = X_full.to_numpy()
X_full = X_full.reshape(8000,49,41)

y1 = pd.read_parquet('processed-data/processed-outcomes-b.parquet')
y1 = y1["In-hospital_death"].to_numpy().flatten()
y2 = pd.read_parquet('processed-data/processed-outcomes-b.parquet')
y2 = y2["In-hospital_death"].to_numpy().flatten()

y_full = np.concatenate((y1,y2), axis=0)

full_dataset = PatientDataset(X_full, y_full)
full_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Uncomment/comment line depending on if you want to use normal LSTM or bidirectional LSTM
#model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
model = BidirectionalLSTM(input_size, hidden_size, num_layers, output_size).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop which prints current training loss and validation loss

# NOTE: This is for validation error analysis

total_val_loss = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(train_loader)

    # Validation phase
    model.eval() 
    val_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item()
    
    val_loss /= len(test_loader)
    total_val_loss += val_loss
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

total_val_loss /= num_epochs
print(f"Avg val loss: {total_val_loss:.4f}")

In [58]:
# Training loop which prints current training loss and AUROC/AUPRC

# NOTE: Change train_loader with full_loader if we want to train the model on set a and set b (after validation error analysis)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(full_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(full_loader)

    # AUROC & AUPRC calculation
    model.eval()
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            probs = torch.sigmoid(outputs).squeeze()

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    auroc = roc_auc_score(all_labels, all_probs)
    auprc = average_precision_score(all_labels, all_probs)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test AUROC: {auroc:.4f}, Test AUPRC: {auprc:.4f}")

Epoch [1/10], Loss: 0.4158, Test AUROC: 0.8012, Test AUPRC: 0.4317
Epoch [2/10], Loss: 0.3967, Test AUROC: 0.8170, Test AUPRC: 0.4508
Epoch [3/10], Loss: 0.3901, Test AUROC: 0.8058, Test AUPRC: 0.4282
Epoch [4/10], Loss: 0.3868, Test AUROC: 0.8054, Test AUPRC: 0.4393
Epoch [5/10], Loss: 0.3817, Test AUROC: 0.7882, Test AUPRC: 0.4219
Epoch [6/10], Loss: 0.3780, Test AUROC: 0.7869, Test AUPRC: 0.4187
Epoch [7/10], Loss: 0.3741, Test AUROC: 0.7851, Test AUPRC: 0.4108
Epoch [8/10], Loss: 0.3667, Test AUROC: 0.7724, Test AUPRC: 0.3964
Epoch [9/10], Loss: 0.3633, Test AUROC: 0.7709, Test AUPRC: 0.3684
Epoch [10/10], Loss: 0.3578, Test AUROC: 0.7615, Test AUPRC: 0.3782


### Q2.3a Transformers (3 Pts)

In [23]:
class Transformer(nn.Module):
    def __init__(self, input_size, num_classes, d_model=128, num_heads=8, num_layers=3, dim_feedforward=512, dropout=0.1):
        super(Transformer, self).__init__()
        
        self.embedding = nn.Linear(input_size, d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, # Dimensionality of embeddings
            nhead=num_heads, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout, 
            batch_first=True
        )
        
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # FC layer for classification
        self.fc = nn.Linear(d_model, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        x = self.fc(x).squeeze()
        return x

In [24]:
# Model hyperparameters
input_size = 41
num_classes = 1 
d_model = 128
num_heads = 8
num_layers = 4 
dim_feedforward = 512
dropout = 0.1
learning_rate = 0.0005
num_epochs = 10

# Test AUROC: 0.8338, Test AUPRC: 0.4719 (Only set a)
""" input_size = 41
num_classes = 1 
d_model = 128
num_heads = 16
num_layers = 8
dim_feedforward = 512
dropout = 0.1
learning_rate = 0.0005
num_epochs = 6 """

# Test AUROC: 0.8250, Test AUPRC: 0.4619 (set a and set b)
""" input_size = 41
num_classes = 1 
d_model = 128
num_heads = 8
num_layers = 4 
dim_feedforward = 512
dropout = 0.1
learning_rate = 0.0005
num_epochs = 3 """

' input_size = 41\nnum_classes = 1 \nd_model = 128\nnum_heads = 8\nnum_layers = 4 \ndim_feedforward = 512\ndropout = 0.1\nlearning_rate = 0.0005\nnum_epochs = 3 '

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(input_size, num_classes, d_model, num_heads, num_layers, dim_feedforward, dropout).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([7.0]).to(device))  # Handle class imbalance
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [26]:
# Training loop which prints current training loss and AUROC/AUPRC

# NOTE: Change train_loader with full_loader if we want to train the model on set a and set b (after validation error analysis)

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(full_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(full_loader)

    # Validation phase
    model.eval()
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            probs = torch.sigmoid(outputs).squeeze()

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Compute AUROC & AUPRC
    auroc = roc_auc_score(all_labels, all_probs)
    auprc = average_precision_score(all_labels, all_probs)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test AUROC: {auroc:.4f}, Test AUPRC: {auprc:.4f}")

Epoch [1/10], Loss: 1.2523, Test AUROC: 0.7984, Test AUPRC: 0.4489
Epoch [2/10], Loss: 1.2172, Test AUROC: 0.8065, Test AUPRC: 0.4481
Epoch [3/10], Loss: 1.1960, Test AUROC: 0.8282, Test AUPRC: 0.4854
Epoch [4/10], Loss: 1.1841, Test AUROC: 0.8329, Test AUPRC: 0.4898
Epoch [5/10], Loss: 1.1674, Test AUROC: 0.8238, Test AUPRC: 0.4902
Epoch [6/10], Loss: 1.1504, Test AUROC: 0.8055, Test AUPRC: 0.4630
Epoch [7/10], Loss: 1.1386, Test AUROC: 0.8094, Test AUPRC: 0.4363
Epoch [8/10], Loss: 1.1076, Test AUROC: 0.7965, Test AUPRC: 0.4420
Epoch [9/10], Loss: 1.0773, Test AUROC: 0.7908, Test AUPRC: 0.4202
Epoch [10/10], Loss: 1.0246, Test AUROC: 0.7489, Test AUPRC: 0.3922


### Q2.3b Tokenizing Time-Series Data and Transformers (4 Pts)

In [63]:
# Time conversion from "XX:YY" to an interval between [0,1]
def scale_time(time_str):
    # Split the time string into hours and minutes
    hours, minutes = map(int, time_str.split(":"))
    
    # Convert the time to total minutes
    total_minutes = hours * 60 + minutes
    
    # Maximum time is 48 hours (2880 minutes)
    max_time = 48 * 60
    
    # Scale time to the range [0, 1]
    scaled_time = total_minutes / max_time
    return scaled_time

In [64]:
# One-Hot Encoding of the different categories
categories = [['ALP', 'ALT', 'AST', 'Age', 'Albumin', 'BUN', 'Bilirubin', 'Cholesterol', 'Creatinine', 'DiasABP', 'FiO2',
                'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'HR', 'Height', 'K', 'Lactate', 'MAP', 'MechVent', 'Mg', 'NIDiasABP',
                'NIMAP', 'NISysABP', 'Na', 'PaCO2', 'PaO2', 'Platelets', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI',
                'TroponinT', 'Urine', 'WBC', 'Weight', 'pH']]
encoder = OneHotEncoder(categories=categories, sparse_output=False)

In [65]:
def scale_values(df):
    # Ensure the "Value" column is numeric
    df = df[pd.to_numeric(df["Value"], errors="coerce").notna()].copy()
    df["Value"] = df["Value"].astype(float)

    # Apply StandardScaler to the "Value" column
    scaler = StandardScaler()
    df["Value"] = scaler.fit_transform(df[["Value"]])

    return df

In [66]:
# Getting max_sequence_length (= 1500, needed for padding the sequences). Note that set-a contains the largest sequence
file_paths = glob.glob("data/set-a/*.txt")
max_sequence_length = 0

for file in file_paths:
    df = pd.read_csv(file)
    df = df[df['Parameter'] != 'ICUType']
    df = df[df['Parameter'] != 'RecordID']
    if(len(df) > max_sequence_length):
        max_sequence_length = len(df)

# Experimental: Clip max_sequence_length to 1000
max_sequence_length = 1000
print(max_sequence_length)

1000


In [67]:
# Processing all sequences and appending them to a list for extracting the embeddings

time_sequences_a = []
category_sequences_a = []
value_sequences_a = []

time_sequences_b = []
category_sequences_b = []
value_sequences_b = []

time_sequences_c = []
category_sequences_c = []
value_sequences_c = []


# Set-a processing
file_paths = glob.glob("data/set-a/*.txt")
i = 0
for file in file_paths:
    df = pd.read_csv(file)

    # Filter out "ICUType" & "RecordID"
    df = df[df['Parameter'] != 'ICUType']
    df = df[df['Parameter'] != 'RecordID']

    # Time scaling
    df["Time"] = df["Time"].apply(scale_time).astype(float)

    # Parameter one-hot encoding
    parameter_column = df["Parameter"].values.reshape(-1, 1)
    encoded_params = encoder.fit_transform(parameter_column)
    indices = np.argmax(encoded_params, axis=1)
    df["Parameter"] = indices

    # Scaling the values
    df = scale_values(df)
    
    time_sequences_a.append(torch.tensor(df["Time"].to_numpy()))
    category_sequences_a.append(torch.tensor(df["Parameter"].to_numpy()))
    value_sequences_a.append(torch.tensor(df["Value"].to_numpy()))

print("Processing of Set-a done")    


# Set-b processing
file_paths = glob.glob("data/set-b/*.txt")
i = 0
for file in file_paths:
    df = pd.read_csv(file)

    # Filter out "ICUType" & "RecordID"
    df = df[df['Parameter'] != 'ICUType']
    df = df[df['Parameter'] != 'RecordID']

    # Time scaling
    df["Time"] = df["Time"].apply(scale_time).astype(float)

    # Parameter one-hot encoding
    parameter_column = df["Parameter"].values.reshape(-1, 1)
    encoded_params = encoder.fit_transform(parameter_column)
    indices = np.argmax(encoded_params, axis=1)
    df["Parameter"] = indices

    # Scaling the values
    df = scale_values(df)
    
    time_sequences_b.append(torch.tensor(df["Time"].to_numpy()))
    category_sequences_b.append(torch.tensor(df["Parameter"].to_numpy()))
    value_sequences_b.append(torch.tensor(df["Value"].to_numpy()))

print("Processing of Set-b done")


# Set-c processing
file_paths = glob.glob("data/set-c/*.txt")
i = 0
for file in file_paths:
    df = pd.read_csv(file)

    # Filter out "ICUType" & "RecordID"
    df = df[df['Parameter'] != 'ICUType']
    df = df[df['Parameter'] != 'RecordID']
    df = df[df['Parameter'] != '']

    # Remove nans in parameter column
    df = df.dropna(subset=["Parameter"])

    # Time scaling
    df["Time"] = df["Time"].apply(scale_time).astype(float)

    # Parameter one-hot encoding
    parameter_column = df["Parameter"].values.reshape(-1, 1)
    encoded_params = encoder.fit_transform(parameter_column)
    indices = np.argmax(encoded_params, axis=1)
    df["Parameter"] = indices

    # Scaling the values
    df = scale_values(df)
    
    time_sequences_c.append(torch.tensor(df["Time"].to_numpy()))
    category_sequences_c.append(torch.tensor(df["Parameter"].to_numpy()))
    value_sequences_c.append(torch.tensor(df["Value"].to_numpy()))

print("Processing of Set-c done") 

Processing of Set-a done
Processing of Set-b done
Processing of Set-c done


In [68]:
num_variables = 40
embedding_dim = 20

# Embedding layer for categorical/parameter variables
variable_embedding = nn.Embedding(num_variables, embedding_dim)

# Linear layer to project (time, value) into the same space
time_value_proj = nn.Linear(2, embedding_dim)

In [69]:
def process_patient_data(time_sequences, category_sequences, value_sequences):
    # Convert categorical variables to embeddings
    var_embeddings = [variable_embedding(seq) for seq in category_sequences]

    # Stack and project time and value
    time_value_seqs = [torch.cat([t.unsqueeze(1), v.unsqueeze(1)], dim=1).to(torch.float32) for t, v in zip(time_sequences, value_sequences)]
    time_value_embeddings = [time_value_proj(seq) for seq in time_value_seqs]

    # Concatenate everything
    final_sequences = [torch.cat([var_emb, tv_emb], dim=1) 
                       for var_emb, tv_emb in zip(var_embeddings, time_value_embeddings)]

    # Pad sequences to a fixed length of max_sequence_length
    max_len = max_sequence_length
    padded_sequences = torch.zeros(len(final_sequences), max_len, final_sequences[0].size(1))

    for i, seq in enumerate(final_sequences):
        length = min(seq.size(0), max_len)
        padded_sequences[i, :length, :] = seq[:length]
    
    return padded_sequences

In [70]:
# Getting embeddings which are the inputs of the transformer
padded_sequences_a = process_patient_data(time_sequences_a, category_sequences_a, value_sequences_a)
padded_sequences_b = process_patient_data(time_sequences_b, category_sequences_b, value_sequences_b)
padded_sequences_c = process_patient_data(time_sequences_c, category_sequences_c, value_sequences_c)
print(padded_sequences_a.shape)
print(padded_sequences_b.shape)
print(padded_sequences_c.shape)

torch.Size([4000, 1000, 40])
torch.Size([4000, 1000, 40])
torch.Size([4000, 1000, 40])


In [71]:
class PatientSequences(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings.clone().detach().float()
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

def get_dataloader(embeddings, labels, batch_size=32, shuffle=True):
    dataset = PatientSequences(embeddings, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [72]:
X = padded_sequences_a
y = pd.read_parquet('processed-data/processed-outcomes-a.parquet')
y = y["In-hospital_death"].to_numpy().flatten()

train_dataset = PatientSequences(X, y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

X = padded_sequences_b
y = pd.read_parquet('processed-data/processed-outcomes-b.parquet')
y = y["In-hospital_death"].to_numpy().flatten()

val_dataset = PatientSequences(X, y)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

X = padded_sequences_c
y = pd.read_parquet('processed-data/processed-outcomes-c.parquet')
y = y["In-hospital_death"].to_numpy().flatten()

test_dataset = PatientSequences(X, y)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [73]:
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=max_sequence_length):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.shape[1], :]

class PreNormTransformerLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x, mask=None):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x), attn_mask=mask, need_weights=False)[0]
        x = x + self.ff(self.norm2(x))
        return x

class Transformer(nn.Module):
    def __init__(self, input_dim=50, nhead=4, num_layers=3, dim_feedforward=128, dropout=0.2, max_sequence_length=max_sequence_length):
        super(Transformer, self).__init__()
        
        self.positional_encoding = SinusoidalPositionalEncoding(input_dim, max_sequence_length)
        
        self.layers = nn.ModuleList([
            PreNormTransformerLayer(input_dim, nhead, dim_feedforward, dropout) for _ in range(num_layers)
        ])

        self.fc = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.positional_encoding(x)
        for layer in self.layers:
            x = layer(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return self.sigmoid(x)

In [74]:
# Hyperparameters
input_dim = 40
hidden_dim = 256         
num_heads = 4          
num_layers = 4        
learning_rate = 0.0005
num_epochs = 10

# Test AUROC: 0.7714, Test AUPRC: 0.3392
""" input_dim = 40
hidden_dim = 256         
num_heads = 4          
num_layers = 4        
learning_rate = 0.0005
num_epochs = 11 """

' input_dim = 40\nhidden_dim = 256         \nnum_heads = 4          \nnum_layers = 4        \nlearning_rate = 0.0005\nnum_epochs = 11 '

In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(input_dim, num_heads, num_layers).to(device)
# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop which prints current training loss and validation loss

# NOTE: This is for validation error analysis

total_val_loss = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(train_loader)

    # Validation phase
    model.eval() 
    val_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item()
    
    val_loss /= len(test_loader)
    total_val_loss += val_loss
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

total_val_loss /= num_epochs
print(f"Avg val loss: {total_val_loss:.4f}")

In [76]:
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_loss = running_loss / len(train_loader)

    # Validation phase
    model.eval()
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            probs = torch.sigmoid(outputs).squeeze()

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Compute AUROC & AUPRC
    auroc = roc_auc_score(all_labels, all_probs)
    auprc = average_precision_score(all_labels, all_probs)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test AUROC: {auroc:.4f}, Test AUPRC: {auprc:.4f}")

Epoch [1/10], Loss: 0.4348, Test AUROC: 0.6171, Test AUPRC: 0.2191
Epoch [2/10], Loss: 0.3997, Test AUROC: 0.6474, Test AUPRC: 0.2316
Epoch [3/10], Loss: 0.3924, Test AUROC: 0.6948, Test AUPRC: 0.2775
Epoch [4/10], Loss: 0.3872, Test AUROC: 0.7152, Test AUPRC: 0.2753
Epoch [5/10], Loss: 0.3760, Test AUROC: 0.7343, Test AUPRC: 0.2852
Epoch [6/10], Loss: 0.3680, Test AUROC: 0.7616, Test AUPRC: 0.3350
Epoch [7/10], Loss: 0.3517, Test AUROC: 0.7664, Test AUPRC: 0.3509
Epoch [8/10], Loss: 0.3495, Test AUROC: 0.7637, Test AUPRC: 0.3498
Epoch [9/10], Loss: 0.3486, Test AUROC: 0.7684, Test AUPRC: 0.3532
Epoch [10/10], Loss: 0.3398, Test AUROC: 0.7699, Test AUPRC: 0.3624
